# Project#1 of AI Course of University of Tehran
Rasta Tadayon 810196436

***
<h2 align = 'center'>Informed and Unimfored Search</h2>

Main purpose of this project is to use informed and uninformed searches to find the solution for at hand problem. The problem we're facing here is finding the optimal way of getting patients to hospitals using one ambulance.

The map is given to us through a text file. Patients are shown as P characters, Hospitals are shown as numbers which indicate the capacity of that hospital, character A represents the ambulance location in the map. # characters represent walls. Something to note is that the ambulance does not pick up the patients, rather it pushes them around in the map to get them to hospital.

In [37]:
import math
import numpy as np
import time
import heapq
import copy
from texttable import Texttable # used to print in tabular form

### Reading map files

In [2]:
f1 = open("testcases/test1.txt", "r")
mapStr1 = f1.read()
f2 = open("testcases/test2.txt", "r")
mapStr2 = f2.read()
f3 = open("testcases/test3.txt", "r")
mapStr3 = f3.read()

We use three map files which are like the following and are given to the program as a text file. The **#** signs represent walls. **P** characters represent the patients in the map, the numbers represent the hospitals or better said, the capacity of the hospital and the **A** character represent the ambulence (rather its coardinates) in the map.

In [8]:
print "map #1:\n", mapStr1, "\n\n-------------"
print "\nmap #2:\n", mapStr2, "\n\n-------------"
print "\nmap #3:\n", mapStr3

map #1:
######
#  A #
# #P #
#    #
#P   #
#2   #
###### 

-------------

map #2:
######
#   3#
# ## #
#   P#
# PP #
#1P A#
###### 

-------------

map #3:
##########
#    3P  #
#       A#
## #### ##
#   P P  #
#        #
##########


In [9]:
AMB = 'A'
HOSP = 'H'
PATN = 'P'

In [19]:
def getBaseMap(mapStr):
    res = ''
    for i in range(len(mapStr)):
        if(mapStr[i] != '#' and mapStr[i] != ' ' and mapStr[i] !='\n'):
            res += ' '
        else:
            res += mapStr[i]
    return res

baseMap1 = getBaseMap(mapStr1).split('\n')
baseMap2 = getBaseMap(mapStr2).split('\n')
baseMap3 = getBaseMap(mapStr3).split('\n')
map1 = mapStr1.split('\n')
map2 = mapStr2.split('\n')
map3 = mapStr3.split('\n')

Base of all maps are kept as global values to indicate where the static elements are(meaning empty spaces and walls).

In [20]:
def getInitLocs(mapStr):
    ambLoc = []
    patientsLoc = []
    hospLocs = dict()
    for i in range(len(mapStr)):
        for j in range(len(mapStr[0])):
            if(mapStr[i][j] == AMB):
                ambLoc = [i,j]
            if(mapStr[i][j] == PATN):
                patientsLoc.append([i,j])
            if(mapStr[i][j].isdigit()):
                hospLocs[i,j] = int(mapStr[i][j])
    return ambLoc, patientsLoc, hospLocs

The function above gives the initial placement of elements of the map. Patients are kept in a list and hospitals are kept in a dictionary where the coordinates are mapped to the capacity of that hospital.

In [21]:
initAmbLoc1, initPatientsLoc1, initHospLoc1 = getInitLocs(map1)
initAmbLoc2, initPatientsLoc2, initHospLoc2 = getInitLocs(map2)
initAmbLoc3, initPatientsLoc3, initHospLoc3 = getInitLocs(map3)

## Uninformed Search

For the uninformed searches we consider all possible moves the agent which here is an ambulance can make and search between all of those cases to find the solution. The uninformed searches used in this project are **BFS(Breadth First Search)** and **IDS(Iterative Deepening Search)**. In order to keep the states we have a state class as follows. States are distinguished from one another by the location of ambulance, patients and capacities of hospitals.


In order to correctly understand and solve the problem it should be modeled carefully. This project has the following elements and is modeled as will be explained:

- __State__: Each state is defined as is differentiated from the other states by the current situation of map and elements inside it (meaning location of ambulance and each patient and capacities of the hospitals)
- __Initial state__: Initial state is the first node and the first state, containing the initial positioning of the ambulance and patients and also the initial capacities of the hospitals.
- __Goal state__: Goal state is reached when all the patients are in hospitals and since after patients disapear after reaching the hospitals, goal state in other word is reached when there are no more patients in the map.
- __Actions__ : Actions are the moves the ambulance can make which can be chosen from the four directions: up, down, right, left. After each action is taken changes happen in the coordinates of different elements of the map which changes the state.

The elements explained above are generalizable to the uninformed search algorithms as well.

In [22]:
class State:
    def __init__(self, ambLoc, patntLoc, hospLoc, parent = None):
        self.aLoc = ambLoc
        self.pLoc = patntLoc
        self.hLoc = hospLoc
        if (parent == None):
            self.path = ''
        else:
            self.path = parent.path
        self.parent = parent
    
    def getHashed(self):
        return str([self.aLoc, self.pLoc, self.hLoc])
    
    def ifDone(self):
        if (len(self.pLoc) == 0):
            return True
        return False
    
    def getFeasibleMoves(self, num):
        i = self.aLoc[0]
        j = self.aLoc[1]
        moves = []
        if (num == 1):
            baseMap = baseMap1
        elif(num == 2):
            baseMap = baseMap2
        elif(num == 3):
            baseMap = baseMap3
        #up 
        if(baseMap[i-1][j] != '#'):
            if (not([i-1,j] in self.pLoc and ([i-2,j] in self.pLoc or baseMap[i-2][j] == '#'))):
                moves.append('u')
        #down
        if(baseMap[i+1][j] != '#'):
            if (not([i+1,j] in self.pLoc and ([i+2,j] in self.pLoc or baseMap[i+2][j] == '#'))):
                moves.append('d')
        #right
        if(baseMap[i][j+1] != '#'):
            if (not([i,j+1] in self.pLoc and ([i,j+2] in self.pLoc or baseMap[i][j+2] == '#'))):
                moves.append('r')
        #left
        if(baseMap[i][j-1] != '#'):
            if (not([i,j-1] in self.pLoc and ([i,j-2] in self.pLoc or baseMap[i][j-2] == '#'))):
                moves.append('l')
        return moves
    
    def moveRight(self):
        i = self.aLoc[0]
        j = self.aLoc[1]
        self.aLoc = [i, j+1]
        #if right side is empty ambulance just moves right

        #if right side is a Patient but no hospital in front of it
        if([i,j+1] in self.pLoc):
            if((i,j+2) in self.hLoc):
                self.pLoc.remove([i,j+1])
                hospCapacity = self.hLoc[(i,j+2)]
                hospCapacity -= 1
                if(hospCapacity):
                    self.hLoc[(i,j+2)] = hospCapacity
                else:
                    del self.hLoc[(i,j+2)]
            else:
                self.pLoc.remove([i,j+1])
                self.pLoc.append([i,j+2])
                self.pLoc.sort()
        self.path = self.path + 'r'
                
    def moveLeft(self):
        i = self.aLoc[0]
        j = self.aLoc[1]
        self.aLoc = [i, j-1]
        #if left side is empty ambulance just moves left

        #if left side is a Patient but no hospital in front of it
        if([i,j-1] in self.pLoc):
            if((i,j-2) in self.hLoc):
                self.pLoc.remove([i,j-1])
                hospCapacity = self.hLoc[(i,j-2)]
                hospCapacity -= 1
                if(hospCapacity):
                    self.hLoc[(i,j-2)] = hospCapacity
                else:
                    del self.hLoc[(i,j-2)]
            else:
                self.pLoc.remove([i,j-1])
                self.pLoc.append([i,j-2])
                self.pLoc.sort()
        self.path = self.path + 'l'
                
    def moveUp(self):
        i = self.aLoc[0]
        j = self.aLoc[1]
        self.aLoc = [i-1, j]
        #if left side is empty ambulance just moves left

        #if left side is a Patient but no hospital in front of it
        if([i-1,j] in self.pLoc):
            if((i-2,j) in self.hLoc):
                self.pLoc.remove([i-1,j])
                hospCapacity = self.hLoc[(i-2,j)]
                hospCapacity -= 1
                if(hospCapacity):
                    self.hLoc[(i-2,j)] = hospCapacity
                else:
                    del self.hLoc[(i-2,j)]
            else:
                self.pLoc.remove([i-1,j])
                self.pLoc.append([i-2,j])
                self.pLoc.sort()
        self.path = self.path + 'u'
    def moveDown(self):
        i = self.aLoc[0]
        j = self.aLoc[1]
        self.aLoc = [i+1, j]
        #if left side is empty ambulance just moves left

        #if left side is a Patient but no hospital in front of it
        if([i+1,j] in self.pLoc):
            if((i+2,j) in self.hLoc):
                self.pLoc.remove([i+1,j])
                hospCapacity = self.hLoc[(i+2,j)]
                hospCapacity -= 1
                if(hospCapacity):
                    self.hLoc[(i+2,j)] = hospCapacity
                else:
                    del self.hLoc[(i+2,j)]
            else:
                self.pLoc.remove([i+1,j])
                self.pLoc.append([i+2,j])
                self.pLoc.sort()
        self.path = self.path + 'd'

Each state can return the feasible moves from that state and can move on to those states with the move functions.

## BFS

We are implementing Breadth First Search on a graph which has loops. To avoid infinite loops we need to keep track of the visited nodes. For that reason we have made the state class. The goal is to find the goal state.


The **BFS** algorithm itself goes as follows:
We have a queue (FIFO) to store the nodes (states) we want to visit next as the frontier and also we have a set of nodes that were visited.

- Firstly the initial state is added to the queue.
- Then all possible moves from there are added to the frontier on the condition that they were not previously visited.
- This sequence continues until either the goal state is reached or all the nodes are visited.

**BFS** is an optimal uninformed search algorithm which mean it will return the optimal (least steps required) answer to our problem.

In [28]:
def bfs(initState,testCaseNum):
    queue = []
    explored = set()
    totalNumOfExploredStates = 0
    totalMoves = 0
    node = initState
    if(node.ifDone()):
        return totalMoves, totalNumOfExploredStates+1, node.path, node
    else:
        hashedState = node.getHashed()
        if(hashedState not in explored):
            queue.append(node)
            explored.add(hashedState)
            while queue:
                node = queue.pop(0)
                totalNumOfExploredStates += 1
                moves = node.getFeasibleMoves(testCaseNum)
                for move in moves:
                    newNode = State(copy.copy(node.aLoc), copy.copy(node.pLoc), copy.copy(node.hLoc), node)
                    
                    if(move == 'r'):
                        newNode.moveRight()
                    elif(move == 'l'):
                        newNode.moveLeft()
                    elif(move == 'u'):
                        newNode.moveUp()
                    elif(move == 'd'):
                        newNode.moveDown()                    
                    
                    if(newNode.ifDone()):
                        return totalMoves, totalNumOfExploredStates, newNode.path, newNode
                    if( not (newNode.getHashed() in explored)):
                        explored.add(newNode.getHashed())
                        queue.append(newNode)

In **BFS** we only visit each state once, therefore the total states explored is equal to the total unique satates explored, because only the nodes in the frontier are explored and the nodes are added to the frontier only if they have not been explored before or in other words have not been added to the explored set yet.

The algorithm is run 3 times for each test case and the mean of the 3 run times is as follows:

In [29]:
def runBFS(node, testCaseNum):
    duration = 0
    for i in range(3):
        start = time.time()
        totalMoves, totalExplored, path, finishNode = bfs(node, testCaseNum)
        end = time.time()
        duration += (end - start)
    return totalExplored, finishNode, duration/3

In [30]:
initNodeBFS1 = State(initAmbLoc1, initPatientsLoc1, initHospLoc1)
initNodeBFS2 = State(initAmbLoc2, initPatientsLoc2, initHospLoc2)
initNodeBFS3 = State(initAmbLoc3, initPatientsLoc3, initHospLoc3)

In [31]:
totalExp1, finishNode1, duration1 = runBFS(initNodeBFS1, 1)
totalExp2, finishNode2, duration2 = runBFS(initNodeBFS2, 2)
totalExp3, finishNode3, duration3 = runBFS(initNodeBFS3, 3)

In [40]:
tableBFS = [['BFS', 'Duration', 'Total Number of Moves','Route', 'Total Explored States'],
            ['Testcase#1', duration1, len(finishNode1.path), finishNode1.path, totalExp1],
            ['Testcase#2', duration2, len(finishNode2.path), finishNode2.path, totalExp2],
            ['Testcase#3', duration3, len(finishNode3.path), finishNode3.path, totalExp3]]


In [41]:
t = Texttable()
t.add_rows(tableBFS)
print(t.draw())

+------------+----------+------------------+-----------------+-----------------+
|    BFS     | Duration | Total Number of  |      Route      | Total Explored  |
|            |          |      Moves       |                 |     States      |
+============+==========+==================+=================+=================+
| Testcase#1 | 0.009    | 11               | dddrdlluuld     | 285             |
+------------+----------+------------------+-----------------+-----------------+
| Testcase#2 | 0.213    | 27               | uuudddllurdruuu | 8710            |
|            |          |                  | ulllddrrdruu    |                 |
+------------+----------+------------------+-----------------+-----------------+
| Testcase#3 | 0.603    | 39               | ulldrdddllurdru | 26149           |
|            |          |                  | uurulldrddlllld |                 |
|            |          |                  | luuulurrr       |                 |
+------------+----------+---

The paths for each test case and number of moves are stated above. The duration column specifies the time it took for the algorithm to find the path for the ambulence to take all the patients to the hospitals.

## IDS

**Iterative Deepening Search** is also an uninformed search algorithm. **IDS** actually implements DFS with different depths, meaning it runs DFS with a limit for the depth so basically IDS is doing DFS in a BFS manner. 

In **IDS** it is possible to see a repeated state but in less depth, therfore a new depth value is also needed to indicate the state of a node, so in the new class created for IDS search we also add a depth attribute as will be shown in the following.

The **IDS** algorithm uses a stack (LIFO) to keep track of the explored states. The algorithm is as follows:

- In the outer loop we increase the depth of which the dfs will run in and run the dfs with the given depth.
- In every iteration the explored set and the stack will get empties (this is the reason why this algorithm may visit already visited nodes.
- For the limited DFS:
    - Pop the top node from the stack.
    - If that node has the depth equal to the depth we are already at its children can not be explored for they have larger depths.
    - If not the children are added to the stack and these steps will continue until either the goal node is found or the algorithm ends.

In [42]:
class idsState( State ):            
        def __init__(self, ambLoc, patntLoc, hospLoc, parent = None, nodeDepth = 0): 
                self.depth = nodeDepth  
                State.__init__(self, ambLoc, patntLoc, hospLoc, parent)
        def getHashed(self):
            return str([self.aLoc, self.pLoc, self.hLoc, self.depth])

In [47]:
maxDepth1 = len(baseMap1)*len(baseMap1[0])
maxDepth2 = len(baseMap2)*len(baseMap2[0])
maxDepth3 = len(baseMap3)*len(baseMap3[0])

def ids(initState, testNum):
    if(testNum == 1):
        maxDepth = maxDepth1
    elif(testNum == 2):
        maxDepth = maxDepth2
    elif(testNum == 3):
        maxDepth = maxDepth3
    uniqueStates = set()
    totalNumOfStates = 0
    totalUniqueStates = 1
    if(initState.ifDone()):
        return totalNumOfStates , totalUniqueStates, node
    else:
        for i in range(1, maxDepth+1):
            stack = []
            explored = set()
            hashed = initState.getHashed()
            if(not(hashed in explored)):
                stack.append(initState)
                explored.add(hashed)
                totalNumOfStates += 1
                if(hashed not in uniqueStates):
                    uniqueStates.add(hashed)
                    totalUniqueStates += 1
            while stack:
                node = stack.pop()
                if(node.ifDone()):
                    return totalNumOfStates, totalUniqueStates, node
                if(node.depth == i):
                    continue
                moves = node.getFeasibleMoves(testNum)
                for move in moves:
                    newNode = idsState(copy.copy(node.aLoc), copy.copy(node.pLoc), copy.copy(node.hLoc), node, (node.depth) + 1)
                    if(move == 'r'):
                        newNode.moveRight()
                    elif(move == 'l'):
                        newNode.moveLeft()
                    elif(move == 'u'):
                        newNode.moveUp()
                    elif(move == 'd'):
                        newNode.moveDown()
                    
                    newHashed = newNode.getHashed()
                    if(newHashed not in explored):
                        explored.add(newHashed)
                        stack.append(newNode)
                        totalNumOfStates += 1
                        if(newHashed not in uniqueStates):
                            totalUniqueStates += 1
                            uniqueStates.add(newHashed)                    

In [48]:
def runIDS(node, testCaseNum):
    duration = 0
    for i in range(3):
        start = time.time()
        totalState, totalUnique, finishNode = ids(node, testCaseNum)
        end = time.time()
        duration += (end - start)
    return totalState, totalUnique, finishNode, duration/3

In [49]:
initIDSNode1 = idsState(initAmbLoc1,initPatientsLoc1,initHospLoc1, None, 0)
initIDSNode2 = idsState(initAmbLoc2,initPatientsLoc2,initHospLoc2, None, 0)
initIDSNode3 = idsState(initAmbLoc3,initPatientsLoc3,initHospLoc3, None, 0)

In [50]:
totalStat1, totalUnique1, finishNodeIDS1, durationIDS1 = runIDS(initIDSNode1, 1)
totalStat2, totalUnique2, finishNodeIDS2, durationIDS2 = runIDS(initIDSNode2, 2)
totalStat3, totalUnique3, finishNodeIDS3, durationIDS3 = runIDS(initIDSNode3, 3)


In [52]:
tableIDS = [['IDS', 'Duration', 'Total Number of Moves','Route', 'Total Explored States', 'Total Unique Explored States'],
            ['Testcase#1', durationIDS1, len(finishNode1.path), finishNode1.path, totalStat1, totalUnique1],
            ['Testcase#2', durationIDS2, len(finishNode2.path), finishNode2.path, totalStat2, totalUnique2],
            ['Testcase#3', durationIDS3, len(finishNode3.path), finishNode3.path, totalStat3, totalUnique3]]
t = Texttable()
t.add_rows(tableIDS)
print(t.draw())

+------------+----------+-------------+-------------+-------------+------------+
|    IDS     | Duration |    Total    |    Route    |    Total    |   Total    |
|            |          |  Number of  |             |  Explored   |   Unique   |
|            |          |    Moves    |             |   States    |  Explored  |
|            |          |             |             |             |   States   |
+============+==========+=============+=============+=============+============+
| Testcase#1 | 0.062    | 11          | dddrdlluuld | 2927        | 954        |
+------------+----------+-------------+-------------+-------------+------------+
| Testcase#2 | 5.161    | 27          | uuudddllurd | 292824      | 45306      |
|            |          |             | ruuuulllddr |             |            |
|            |          |             | rdruu       |             |            |
+------------+----------+-------------+-------------+-------------+------------+
| Testcase#3 | 21.244   | 39

The route for each test case along with total number of states visited and number of unique states visited is stated above.

**IDS** is also an optimal search algorithm however it took longer than **BFS** to find the solution because it visits nodes more than once(unless solution is in the first depth). IDS is a good algorithm for when the actions that can be taken are a lot.

## Informed Search

As stated before, in uninformed search strategies all of the possible moves are taken into consideration and they all have the same importance and priority. In **Informed Search** however the algorithm is given hints about the desirability of different states and an evaluation function is used to sort nodes in the way to select the most promising node for expantion in the next interation.


Informed search strategies are usually faster than uninformed ones because they have an idea of which way they should expand the search however the chioce of heuristics becomes very important in informed searches.

One of the **informed search** methods is called **A*** which will be used in this project.

## A* Search Strategy

In this search method we are trying to avoid the expansion of paths which are expensive already. To do so we have a heuristic function. Two heuristics were used in this projects. 

### Fisrt heuristic:

- Summation of the minimum manhatan distances from all patients to their nearest hospitals added to the length of the path already traversed.

### Why this heuristic is admissible:

We want to take every patient to the hospitals and preferably to the nearest hospital. An **admissible heuristic** is one that does not overestimate the cost to reach the goal and it's rather optimistic. Here we know that the least amount of cost is when each patient is delivered to its closest hospital however this might not always happen, meaning the ambulance may have to go around the patients and many other things like that. Therefore this heuristic is for each node an optimistic one because it is calculating the summation of manhatan distances (straight line distances).

We are running this algorithm of a graph rather than a tree and if we want to not visit states repeatedly in a graph the heuristic has to also be **consistant** in order to give the optimal solution.

### Why this heuristic is consistant:

By defenition we know a heuristic is consistant if the following equation holds:

$ cost(A to C) \geq h(A) - h(C)$

Which means if we have statec $C$ after state $A$ the real cost of traversing to state $C$ from state $A$ is larger than the cost implied by the heuristics. Let's check if this equation holds:

For each traversal between the nodes the cost is equal to one so we need to check for every possible move the difference of the heuristics is less than or equal to one:

- Ambulance moves without moving any patients:
    In this case the heusitics for state C and A are equal therfore the difference is equal to zero and the quation holds.
    
    
- Ambulance moves a patient closer to a hospital:
    In this case the the patient moves one step closer so the value will also be less than or equal to one.
    
    
- Ambulance moves a patient further away from the hospital:
    In this case the heuristic of the later state is larger so the value will become negative so the equation holds.
    

A new class is implemented which has the heuristic function and values:

In [53]:
class informedState(State):
    def __init__(self, ambLoc, patntLoc, hospLoc, heuristicNum, parent = None):
        State.__init__(self, ambLoc, patntLoc, hospLoc, parent)
        self.h = heuristicNum
        if(heuristicNum == 1):
            self.heuristicVal = self.calcH1()
        elif(heuristicNum == 2):
            self.heuristicVal = self.calcH2()
    
    def calcH1(self):
        manhatanDist = 0
        for patient in self.pLoc:
            px = patient[0]
            py = patient[1]
            dist = []
            for hosp in self.hLoc:
                hx = hosp[0]
                hy = hosp[1]
                dist.append(abs(px-hx)+abs(py-hy))
            manhatanDist += min(dist) 
        return manhatanDist + len(self.path)
    
    def calcH2(self):
        manhatanDist = 0
        ax = self.aLoc[0]
        ay = self.aLoc[1]
        for patient in self.pLoc:
            px = patient[0]
            py = patient[1]
            dist = []
            for hosp in self.hLoc:
                hx = hosp[0]
                hy = hosp[1]
                dist.append(abs(px-hx)+abs(py-hy))
            manhatanDist += (min(dist) + abs(ax-px) + abs(ay-py))+len(self.path)
        return manhatanDist
    
    def __lt__(self, other):
         return (self.heuristicVal <= other.heuristicVal)

In [54]:
def returnStateBasedOnHash(li, hashedVal):
    for item in li:
        if (item.getHashed() == hashedVal):
            return item
    return None

The **A*** search algorithm is very similar to **BFS** with the difference that a priority queue or a min heap is used to keep the states which we want to explore in the next iterations, meaning the heap keeps the nodes with smaller heurisitic values in a higher proirity. Just like the **BFS** algorithm this one also keeps a set of visited node which won't be visited anymore in the next iterations.

In [55]:
def aStarSearch(initState, hNum, testNum):
    meanHeap = []
    visitedStates = set()
    totalNumOfVisitedStates = 0
    totalMoves = 0
    if(initState.ifDone()):
        return totalNumOfVisitedStates+1, initState
    else:
        hashedState = initState.getHashed()
        if(hashedState not in visitedStates):
            heapq.heappush(meanHeap, initState)
            visitedStates.add(hashedState)
            while meanHeap:
                currState = heapq.heappop(meanHeap)
                totalNumOfVisitedStates += 1
                moves = currState.getFeasibleMoves(testNum)
                for move in moves:
                    newState = informedState(copy.copy(currState.aLoc), copy.copy(currState.pLoc), copy.copy(currState.hLoc), hNum, currState)
                    
                    if(move == 'r'):
                        newState.moveRight()
                    elif(move == 'l'):
                        newState.moveLeft()
                    elif(move == 'u'):
                        newState.moveUp()
                    elif(move == 'd'):
                        newState.moveDown()                    
                    
                    if(newState.ifDone()):
                        return totalNumOfVisitedStates+1, newState
                    if(hNum == 1): newState.heuristicVal = newState.calcH1() 
                    elif(hNum == 2): newState.heuristicVal = newState.calcH2()
                    if( not (newState.getHashed() in visitedStates)):
                        visitedStates.add(newState.getHashed())
                        heapq.heappush(meanHeap, newState)

In [56]:
def runAStar(node, testCaseNum, hNum):
    duration = 0
    for i in range(3):
        start = time.time()
        totalState, finishNode = aStarSearch(node, hNum, testCaseNum)
        end = time.time()
        duration += (end - start)
    return totalState, finishNode, duration/3

In [57]:
initAStarNode1 = informedState(initAmbLoc1,initPatientsLoc1,initHospLoc1, 1)
initAStarNode2 = informedState(initAmbLoc2,initPatientsLoc2,initHospLoc2, 1)
initAStarNode3 = informedState(initAmbLoc3,initPatientsLoc3,initHospLoc3, 1) 

In [58]:
totalStatAStar1, finishNodeAStar1, durationAStar1 = runAStar(initAStarNode1, 1, 1)
totalStatAStar2, finishNodeAStar2, durationAStar2 = runAStar(initAStarNode2, 2, 1)
totalStatAStar3, finishNodeAStar3, durationAStar3 = runAStar(initAStarNode3, 3, 1)

In [62]:
tableAStar = [['A* - Heuristic#1', 'Duration', 'Total Number of Moves','Route', 'Total Explored States'],
            ['Testcase#1', durationAStar1, len(finishNodeAStar1.path), finishNodeAStar1.path, totalStatAStar1],
            ['Testcase#2', durationAStar2, len(finishNodeAStar2.path), finishNodeAStar2.path, totalStatAStar2],
            ['Testcase#3', durationAStar3, len(finishNodeAStar3.path), finishNodeAStar3.path, totalStatAStar3]]
t = Texttable()
t.add_rows(tableAStar)
print(t.draw())

+----------------+----------+----------------+----------------+----------------+
|      A* -      | Duration |  Total Number  |     Route      | Total Explored |
|  Heuristic#1   |          |    of Moves    |                |     States     |
+================+==========+================+================+================+
| Testcase#1     | 0.007    | 11             | dddrdlluuld    | 116            |
+----------------+----------+----------------+----------------+----------------+
| Testcase#2     | 0.191    | 27             | uuudddllurdruu | 3770           |
|                |          |                | udldllurrdruu  |                |
+----------------+----------+----------------+----------------+----------------+
| Testcase#3     | 0.419    | 39             | lulrddddllurdr | 10101          |
|                |          |                | uuurulldrddlll |                |
|                |          |                | ldluuulurrr    |                |
+----------------+----------

As mentioned before since this algorithm is similar to BFS as neither of them visits states more than once therefore only one column represents the number of explores states since the number of explored states is equala to the total unique explored states.

 ### Second heuristic:
 - The secon heuristic consists of the summation of minimum distance of each patient from the closest hospital added with the manhatan distance between the ambulance and patients in each state
 
 This heuristic is also both admissible and consistant.

### Why this heuristic is admissible:

This heuristic adds the manhatan distance between the ambulance and patients with the manhatan distance of each patient from the closest hospital to it. It is obvious that in order to reach the goal state the ambulance must get to each patient and take them to the hospital which means in the best case the ambulance must traverse through the manhatan distance route of patients and take them throught the manhatan distance route to the hospitals. It can be observed that these route may not be available, therefore our heuristic is definitely less than or equal to the cost and is optimistic in each state. 

In [60]:
totalStatAStar1h2, finishNodeAStar1h2, durationAStar1h2 = runAStar(initAStarNode1, 1, 2)
totalStatAStar2h2, finishNodeAStar2h2, durationAStar2h2 = runAStar(initAStarNode2, 2, 2)
totalStatAStar3h2, finishNodeAStar3h2, durationAStar3h2 = runAStar(initAStarNode3, 3, 2)

In [63]:
tableAStar2 = [['A* - Heuristic#2', 'Duration', 'Total Number of Moves','Route', 'Total Explored States'],
            ['Testcase#1', durationAStar1h2, len(finishNodeAStar1h2.path), finishNodeAStar1h2.path, totalStatAStar1h2],
            ['Testcase#2', durationAStar2h2, len(finishNodeAStar2h2.path), finishNodeAStar2h2.path, totalStatAStar2h2],
            ['Testcase#3', durationAStar3h2, len(finishNodeAStar3h2.path), finishNodeAStar3h2.path, totalStatAStar3h2]]
t = Texttable()
t.add_rows(tableAStar2)
print(t.draw())

+----------------+----------+----------------+----------------+----------------+
|      A* -      | Duration |  Total Number  |     Route      | Total Explored |
|  Heuristic#2   |          |    of Moves    |                |     States     |
+================+==========+================+================+================+
| Testcase#1     | 0.005    | 13             | ddlldrurdrdll  | 38             |
+----------------+----------+----------------+----------------+----------------+
| Testcase#2     | 0.008    | 27             | uuudddllurdruu | 157            |
|                |          |                | uulllddrrdruu  |                |
+----------------+----------+----------------+----------------+----------------+
| Testcase#3     | 0.019    | 39             | luldrdddllurdr | 393            |
|                |          |                | uuurulldrddlll |                |
|                |          |                | ldluuulurrr    |                |
+----------------+----------




For both A* searches the heuristics are as said before however the rankings of the priority queue are based on the evaluation function which mean by the value of the heuristic function added with the $g(n)$ function. In this project the $g(n)$ function is considered simply the route passed until current state:

## Table of information:

In [67]:
def tableOfInformation(information):
    testNumber = information['testNumber']
    routeLengths = information['routeLengths']
    totalStates = information['totalStates']
    uniqueStates = information['uniqueStates'] 
    durations = information['durations']
    table = [['Testcase#'+str(testNumber), 'Route Length', 'Total Visited States', 'Total Unique Visited States', 'Duration'],
             ['BFS',routeLengths[0], totalStates[0], uniqueStates[0], durations[0]],
             ['IDS',routeLengths[1], totalStates[1], uniqueStates[1], durations[1]],
             ['A*-h1',routeLengths[2], totalStates[2], uniqueStates[2], durations[2]],
             ['A*-h2',routeLengths[3], totalStates[3], uniqueStates[3], durations[3]]
            ]
    t = Texttable()
    t.add_rows(tableAStar2)
    print(t.draw())

In [ ]:
testCase1Info = {'testNumber' : 1}

In [271]:
print(f'First test case information:\n BFS:    route length : {len(finishNode1.path)} , total nodes visited : {totalExp1} , total unique visited nodes : {totalExp1} , execution time : {duration1}\n IDS:    route length : {len(finishNodeIDS1.path)} , total nodes visited : {totalStat1}, total unique visited nodes : {totalUnique1} , execution time : {durationIDS1}\n A* h1:  route length : {len(finishNodeAStar1.path)} , total nodes visited : {totalStatAStar1}, total unique visited nodes: {totalStatAStar1}, execution time : {durationAStar1}\n A* h2:  route length : {len(finishNodeAStar1h2.path)} , total nodes visited : {totalStatAStar1h2}, total unique visited nodes: {totalStatAStar1h2}, execution time : {durationAStar1h2}\n')

First test case information:
 BFS:    route length : 11 , total nodes visited : 285 , total unique visited nodes : 285 , execution time : 0.010305325190226236
 IDS:    route length : 11 , total nodes visited : 2927, total unique visited nodes : 954 , execution time : 0.04787087440490723
 A* h1:  route length : 11 , total nodes visited : 116, total unique visited nodes: 116, execution time : 0.008270581563313803
 A* h2:  route length : 13 , total nodes visited : 38, total unique visited nodes: 38, execution time : 0.0026597976684570312



In [277]:
print(f'Second test case information:\n BFS:    route length : {len(finishNode2.path)} , total nodes visited : {totalExp2} , total unique visited nodes : {totalExp2} , execution time : {duration2}\n IDS:    route length : {len(finishNodeIDS2.path)} , total nodes visited : {totalStat2}, total unique visited nodes : {totalUnique2} , execution time : {durationIDS2}\n A* h1:  route length : {len(finishNodeAStar2.path)} , total nodes visited : {totalStatAStar2}, total unique visited nodes: {totalStatAStar2}, execution time : {durationAStar2}\n A* h2:  route length : {len(finishNodeAStar2h2.path)} , total nodes visited : {totalStatAStar2h2}, total unique visited nodes: {totalStatAStar2h2}, execution time : {durationAStar2h2}\n')

Second test case information:
 BFS:    route length : 27 , total nodes visited : 8710 , total unique visited nodes : 8710 , execution time : 0.28191224733988446
 IDS:    route length : 27 , total nodes visited : 292824, total unique visited nodes : 45306 , execution time : 6.101008176803589
 A* h1:  route length : 27 , total nodes visited : 3770, total unique visited nodes: 3770, execution time : 0.4568038781483968
 A* h2:  route length : 27 , total nodes visited : 157, total unique visited nodes: 157, execution time : 0.013629833857218424



In [278]:
print(f'Third test case information:\n BFS:    route length : {len(finishNode3.path)} , total nodes visited : {totalExp3} , total unique visited nodes : {totalExp3} , execution time : {duration3}\n IDS:    route length : {len(finishNodeIDS3.path)} , total nodes visited : {totalStat3}, total unique visited nodes : {totalUnique3} , execution time : {durationIDS3}\n A* h1:  route length : {len(finishNodeAStar3.path)} , total nodes visited : {totalStatAStar3}, total unique visited nodes: {totalStatAStar3}, execution time : {durationAStar3}\n A* h2:  route length : {len(finishNodeAStar3h2.path)} , total nodes visited : {totalStatAStar3h2}, total unique visited nodes: {totalStatAStar3h2}, execution time : {durationAStar3h2}\n')

Third test case information:
 BFS:    route length : 39 , total nodes visited : 26149 , total unique visited nodes : 26149 , execution time : 0.8361042340596517
 IDS:    route length : 39 , total nodes visited : 1006734, total unique visited nodes : 141053 , execution time : 29.78260024388631
 A* h1:  route length : 39 , total nodes visited : 10101, total unique visited nodes: 10101, execution time : 1.0528231461842854
 A* h2:  route length : 39 , total nodes visited : 393, total unique visited nodes: 393, execution time : 0.03690107663472494

